In [3]:
import pandas as pd
import math
import numpy as np

X = np.array(
    [
        [1, 1, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 1],
        [0, 1, 1, 0, 1, 0],
        [1, 1, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 0],
        [1, 0, 1, 1, 0, 1],
        [0, 1, 0, 0, 1, 0],
    ]
)

y = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

# Learning


In [ ]:
def entropy(y):
    """
    Computes the entropy of the labels

    Parameters
    ----------
    y: np.ndarray
        Array of labels. eg: [1,0,0,1,1,1,0]

    Returns
    -------
    float
        Entropy value which gives the impurity of the labels

    Notes
    -----
        - Uses the formula: H(y) = -sum(p_i * log2(p_u)) for all classes of i.
        - Zero probs are ignored to avoid log2(0).

    Examples
    --------
    Input: y = [1, 0, 1, 0, 1]
    Step 1: Calculate probabilities
        - Class 0: count = 2, probability = 2/5 = 0.4
        - Class 1: count = 3, probability = 3/5 = 0.6
    Step 2: Apply entropy formula
        - H(y) = -(0.4 * log2(0.4) + 0.6 * log2(0.6))
        - H(y) = -(0.4 * (-1.32) + 0.6 * (-0.74))
        - H(y) = -(-0.528 + (-0.444)) = 0.972
    Output: 0.972 (high entropy indicating mixed classes)
    """
    probs = np.bincount(y) / len(
        y
    )  # bincount counts how many samples are there in each class
    probs = probs[probs > 0]  # removing zero probs
    return -np.sum(probs * np.log2(probs))  # applying the formula


def gini(y):
    probs = np.bincount(y) / len(y)
    return 1 - np.sum(probs**2)


def info_gain(x, y):
    """
    Calculates the information gain of splitting labels y using feature x.

    :param x: np.ndarray
            Array of single feature's values (shape: n_sampels, ).

    :param y: np.ndarray
            Array of target labels (shape: n_samples, ).

    Returns
    -----
    float
        Info gain obtained by splitting y based on feature x

    Notes
    -----
        - Info gain = parent entropy - weighted child entropy
        - Weighted child entropy is calculated based on the proportion of samples
        in each unique feature value.

    Examples
    --------
    Input: x = [1, 0, 1, 0, 1], y = [1, 0, 1, 0, 1]
    Step 1: Calculate parent entropy
        - Parent entropy = entropy([1, 0, 1, 0, 1]) = 0.972
    Step 2: Split by feature values
        - When x = 0: indices [1, 3] → y_child = [0, 0] → entropy = 0.0
        - When x = 1: indices [0, 2, 4] → y_child = [1, 1, 1] → entropy = 0.0
    Step 3: Calculate weighted child entropy
        - Weight for x=0: 2/5 = 0.4, entropy = 0.0
        - Weight for x=1: 3/5 = 0.6, entropy = 0.0
        - Weighted entropy = 0.4 * 0.0 + 0.6 * 0.0 = 0.0
    Step 4: Calculate information gain
        - Info gain = 0.972 - 0.0 = 0.972
    Output: 0.972 (perfect split!)
    """
    parent_entropy = gini(y)  # getting the parent entropy
    values = np.unique(x)  # finding the unique values to split from X
    weighted_entropy = 0
    for v in values:
        # getting the coresponding value of x which is v in y
        y_child = y[v == x]
        weighted_entropy += (len(y_child) / len(y)) * entropy(
            y_child
        )  # using the formula to get the weighted entropy of child

    return (
        parent_entropy - weighted_entropy
    )  # substracting weighted entropy of child from parent entropy to get the information gain


def best_split(x, y):
    """
    Determines the index of the feature that provides the highest information gain.

    Parameters
    ----------
    x : np.ndarray
        Feature matrix of shape (n_samples, n_features).
    y : np.ndarray
        Target labels of shape (n_samples,).

    Returns
    -------
    int
        Index of the feature that gives the maximum information gain.

    Notes
    -----
    - Calls `info_gain` on each feature (column) of x.
    - Returns the feature index corresponding to the largest gain.

    Examples
    --------
    Input: x = [[1,1], [0,1], [1,0], [0,0], [1,1]], y = [1, 0, 1, 0, 1]
    Step 1: Calculate info gain for each feature
        - Feature 0: x[:,0] = [1, 0, 1, 0, 1]
          → info_gain([1, 0, 1, 0, 1], [1, 0, 1, 0, 1]) = 0.972
        - Feature 1: x[:,1] = [1, 1, 0, 0, 1]
          → info_gain([1, 1, 0, 0, 1], [1, 0, 1, 0, 1]) = 0.019
    Step 2: Find maximum gain
        - gains = [0.972, 0.019]
        - argmax(gains) = 0
    Output: 0 (Feature 0 provides the best split)
    """
    gains = [
        info_gain(x[:, i], y) for i in range(x.shape[1])
    ]  # sending all values of single feature and whole y to get the info gain
    return np.argmax(gains)  # returns the index of the largest gain


def build_tree(x, y):
    """
    Recursively builds decision tree using information gain.

    Parameters
    ----------
    x : np.ndarray
        Feature matrix of shape (n_samples, n_features), where each row is a sample and each column is a feature.
    y : np.ndarray
        Target labels of the samples.

    Returns
    -------
    dict or int
        A nested dict representing the decision tree or an integer (0, 1) if a node is a leaf.

    Notes
    -----
    - If all labels in y are the same, returns that label.
    - If no features left to split, returns the majority class.
    - Recursively splits the data based on the feature with the highest info gain.

    Examples
    --------
    Input: x = [[1,1], [0,1], [1,0], [0,0], [1,1]], y = [1, 0, 1, 0, 1]
    Step 1: Check stopping conditions
        - len(np.unique(y)) = 2 (not pure, continue)
        - x.shape[1] = 2 (features available, continue)
    Step 2: Find best split
        - best_split(x, y) returns feature_idx = 0
    Step 3: Create node and split data
        - Node: {"feature": 0, "branches": {}}
        - Split by feature 0 values: [0, 1]
    Step 4: Recursive calls for each branch
        - Branch 0 (x[feature_0] == 0):
          → x_subset = [[0,1], [0,0]], y_subset = [0, 0]
          → All labels same → returns 0
        - Branch 1 (x[feature_0] == 1):
          → x_subset = [[1,1], [1,0], [1,1]], y_subset = [1, 1, 1]
          → All labels same → returns 1
    Output: {"feature": 0, "branches": {0: 0, 1: 1}}
    """

    # if all the lables in the y are same, then its a leaf node. return it.
    if len(np.unique(y)) == 1:
        return int(y[0])

    # if there's no feature left to split, then return the majority class
    if x.shape[1] == 0:
        return int(np.round(np.mean(y)))

    feature_idx = best_split(x, y)  # getting the best feature to split

    tree = {"feature": feature_idx, "branches": {}}  # creating a node

    for value in np.unique(
        x[:, feature_idx]
    ):  # looping through number of values in the splitted feature
        # masking to get the index of splitted values
        idx = X[:, feature_idx] == value
        subtree = build_tree(
            x[idx], y[idx]
        )  # calling build_tree with the splitted values
        tree["branches"][int(value)] = subtree

    return tree


def predict(tree, sample):
    """
    Makes a prediction for a single sample using the trained decision tree.

    Parameters
    ----------
    tree : dict or int
        The decision tree structure returned by build_tree(). Can be:
        - dict: Internal node with 'feature' and 'branches' keys
        - int: Leaf node containing the class prediction (0 or 1)
    sample : np.ndarray or list
        A single sample with feature values to predict.

    Returns
    -------
    int
        Predicted class label (0 or 1).

    Notes
    -----
    - Recursively traverses the tree based on feature values in the sample.
    - At each internal node, uses the sample's feature value to decide which branch to follow.
    - Returns the class label when reaching a leaf node.

    Examples
    --------
    Input: tree = {"feature": 0, "branches": {0: 0, 1: 1}}, sample = [1, 0]
    Step 1: Check if tree is leaf
        - isinstance(tree, int) = False (it's a dict, continue)
    Step 2: Get feature index and sample value
        - feature_idx = tree['feature'] = 0
        - value = sample[0] = 1
    Step 3: Follow the corresponding branch
        - tree['branches'][1] = 1 (this is a leaf node)
    Step 4: Recursive call with leaf node
        - predict(1, sample) → isinstance(1, int) = True → return 1
    Output: 1

    Another example with deeper tree:
    Input: tree = {"feature": 0, "branches": {0: {"feature": 1, "branches": {0: 0, 1: 1}}, 1: 1}},
           sample = [0, 1]
    Step 1: At root node, check feature 0
        - sample[0] = 0 → follow branches[0]
    Step 2: At second node, check feature 1
        - sample[1] = 1 → follow branches[1] = 1 (leaf)
    Step 3: Return leaf value
    Output: 1
    """
    if isinstance(tree, int):
        return tree

    feature_idx = tree["feature"]

    value = sample[feature_idx]

    return predict(tree["branches"][value], sample)

In [5]:
tree = build_tree(X, y)
[predict(tree, sample) for sample in X]

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

In [6]:
X_test = np.array(
    [
        [1, 0, 0, 0, 1, 0],  # free + short → could be spam
        [1, 1, 0, 0, 0, 0],  # free + ! → spam
        [0, 1, 1, 0, 0, 1],  # ! + offer + link → tricky
        [1, 0, 1, 1, 0, 1],  # free + offer + win + link → spam
        [0, 0, 0, 1, 1, 0],  # win + short → could be not spam
        [0, 1, 0, 0, 1, 1],  # ! + short + link → tricky
    ]
)


[predict(tree, sample) for sample in X_test]

[1, 1, 0, 1, 0, 0]

# Practice


In [32]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

In [88]:
# Self


def gini(x):
    classes, counts = np.unique(x, return_counts=True)
    probs = counts / len(x)
    return 1 - np.sum(probs**2)


def info_gain(feature, parent):
    parent_gain = gini(parent)
    best_gain = -1
    best_thresh = None

    unique_values = np.unique(feature)
    if len(unique_values) == 1:
        return 0, unique_values[0]

    thresholds = (unique_values[:-1] + unique_values[1:]) / 2

    for t in thresholds:
        left = parent[feature <= t]
        right = parent[feature >= t]
        weighted_gini = (len(left) / len(parent)) * gini(left) + (
            len(right) / len(parent)
        ) * gini(right)
        gain = parent_gain - weighted_gini
        if gain > best_gain:
            best_gain = gain
            best_thresh = t

    return best_gain, best_thresh


def best_split(feature, parent):
    gains = []
    thresholds = []

    for i in range(feature.shape[1]):
        gain, thresh = info_gain(feature[:, i], parent)
        gains.append(gain)
        thresholds.append(thresh)

    high_gain_idx = np.argmax(gains)
    best_thresh = thresholds[high_gain_idx]
    return high_gain_idx, best_thresh


def build_tree(feature, parent):
    if len(np.unique(parent)) == 1:
        return int(parent[0])

    if feature.shape[1] == 0:
        return int(np.round(np.mean(feature)))

    idx, thresh = best_split(feature, parent)

    tree = {"feature_idx": idx, "threshold": thresh, "branches": {}}

    left_mask = feature[:, idx] <= thresh
    right_mask = feature[:, idx] > thresh
    tree["branches"]["left"] = build_tree(feature[left_mask], parent[left_mask])
    tree["branches"]["right"] = build_tree(feature[right_mask], parent[right_mask])

    return tree


def predict(tree, sample):
    if isinstance(tree, int):
        return tree

    idx = tree["feature_idx"]
    thresh = tree["threshold"]

    if sample[idx] <= thresh:
        return predict(tree["branches"]["left"], sample)
    else:
        return predict(tree["branches"]["right"], sample)


def predict_all(tree, X):
    return np.array([predict(tree, X[i]) for i in range(X.shape[0])])

In [89]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split 
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [90]:
tree = build_tree(X_train, y_train)


In [ ]:
prediction = predict_all(tree, X_test)  
acc = np.sum(prediction == y_test) / len(y_test)
acc

np.float64(0.9308510638297872)

In [105]:
from sklearn import tree

model = tree.DecisionTreeClassifier()

model.fit(X_train, y_train)
sklearn_prediction = model.predict(X_test)

np.sum(prediction == sklearn_prediction) / len(sklearn_prediction)

np.float64(0.9574468085106383)